# MSEE UQ short course:  The $\texttt{UQpy}$ library

Application of Inference using the $\texttt{UQpy}$ module $\texttt{inference}$. 

Detailed instructions on how to use this module can be found in the $\texttt{UQpy}$ documentation.

https://uqpyproject.readthedocs.io/en/latest/inference_doc.html

# Activity 1 

The goal of this activity is to become familiar with various $\texttt{MCMC}$ algorithms

Use the Rosenbrock distribution provided below, and generate samples using the following $\texttt{MCMC}$ algorithms:
- $\texttt{ModifiedMetropolisHastings}$
- $\texttt{DRAM}$
- $\texttt{Stretch}$

Explore the various options provided by each one of the algorithms.

In [ ]:
from UQpy.distributions import DistributionND


class Rosenbrock(DistributionND):
    def __init__(self, p=20.):
        super().__init__(p=p)

    def pdf(self, x):
        return np.exp(-(100 * (x[:, 1] - x[:, 0] ** 2) ** 2 + (1 - x[:, 0]) ** 2) / self.parameters['p'])

    def log_pdf(self, x):
        return -(100 * (x[:, 1] - x[:, 0] ** 2) ** 2 + (1 - x[:, 0]) ** 2) / self.parameters['p']

# Activity 2

Use **information-theoretic** inference to select between probability models. 

You are provided with file $\texttt{'data_ex1b.txt'}$ that contains synthetic measurements for the parameter.  Use model selection with $BIC$ criterion to identify the probability model that best describes the data. 

The set of candidate models consists of the following probability models: 

- $\texttt{Normal}$
- $\texttt{Lognormal}$
- $\texttt{Uniform}$
- $\texttt{ChiSquare}$
- $\texttt{Beta}$
- $\texttt{Levy}$
- $\texttt{InverseGauss}$
- $\texttt{Exponential}$

In [ ]:
from UQpy.inference import DistributionModel, InformationModelSelection, MLE, BIC
from UQpy.distributions import *
import numpy as np

data_ex1b = np.loadtxt('data_ex1b.txt')

m0 = DistributionModel(distributions=Normal(loc=None, scale=None), n_parameters=2, name='normal')
m1 = DistributionModel(distributions=Lognormal(s=None, loc=None, scale=None), n_parameters=3, name='lognormal')
m2 = DistributionModel(distributions=Uniform(loc=None, scale=None), n_parameters=2, name='uniform')
m3 = DistributionModel(distributions=ChiSquare(df=None, loc=None, scale=None), n_parameters=3, name='chi-square')
m4 = DistributionModel(distributions=Beta(a=None, b=None, loc=None, scale=None), n_parameters=4, name='beta')
m5 = DistributionModel(distributions=Levy(loc=None, scale=None), n_parameters=2, name='levy')
m6 = DistributionModel(distributions=InverseGauss(mu=None, loc=None, scale=None), n_parameters=3, name='inv-gauss')
m7 = DistributionModel(distributions=Exponential(loc=None, scale=None), n_parameters=2, name='exponential')

candidate_models = [m0, m1, m2, m3, m4, m5, m6, m7]

mle_list= [MLE(inference_model=model, random_state=0, data=data_ex1b) for model in candidate_models]

selector = InformationModelSelection(parameter_estimators=mle_list, criterion=BIC(), n_optimizations=[5]*8)
selector.sort_models()
print('Sorted model using BIC criterion: ' + ', '.join(m.name for m in selector.candidate_models))
print("Values of model probabilities: ", selector.probabilities)

# Activity 3

Using the **Bayesian model** selection to choose between regression models. Consider the problem of Exercise 1. In this case, each candidate model $m_i$ is given a prior probability $P(m_i) = 1/3$. For each model, our prior belief about its parameters $\theta_i$ is that they are normally distributed as: 
- $m_0(\theta)$: $\theta_0\sim\mathcal{N}(0,10)$.
- $m_1(\boldsymbol{\theta})$: $\theta_0\sim\mathcal{N}(0,1)$, $\theta_1\sim\mathcal{N}(0,1)$
- $m_2(\boldsymbol{\theta})$:  $\theta_0\sim\mathcal{N}(0,1)$, $\theta_1\sim\mathcal{N}(0,2)$, $\theta_2\sim\mathcal{N}(0,0.25)$

Using the same $\texttt{BayesModelSelection}$ of **Exercise 4** of the In-Class exercises

- Change the parameters of the $\texttt{MetropolisHastings}$ sampling, to see how the results change.
- Use $\texttt{ModifiedMetropolisHastings}$, $\texttt{DRAM}$ and $\texttt{MetropolisHastings}$ sampling algorithms for each one of the probability models respectively.  

In [16]:
# Solution
# Solution
import pickle
from UQpy.inference import BayesModelSelection
from UQpy.distributions import Normal, JointIndependent
# Solution
objectRep = open("data_ex3.pkl", "rb")
data3 = pickle.load(objectRep)
objectRep.close()

# Solution
from UQpy.run_model import RunModel, PythonModel

runmodel4 = RunModel(model=PythonModel(model_script='pfn.py', model_object_name='model_linear', var_names=['theta_0']))
runmodel5 = RunModel(model=PythonModel(model_script='pfn1.py', model_object_name='model_quadratic', var_names=['theta_0','theta_1']))
runmodel6 = RunModel(model=PythonModel(model_script='pfn2.py', model_object_name='model_cubic', var_names=['theta_0','theta_1','theta_2']))

# Solution
from UQpy.distributions import JointIndependent, Normal
prior1=Normal(0, 10)
prior2=JointIndependent(marginals=[Normal(0, 1), Normal(0, 1)])
prior3=JointIndependent(marginals=[Normal(0, 1), Normal(0, 2), Normal(0, 0.25)])

# Solution
import numpy as np
from UQpy.inference import ComputationalModel
model_n_params = [1, 2, 3]
model1 = ComputationalModel(n_parameters=1, runmodel_object=runmodel4, prior=prior1,
                            error_covariance=np.ones(50), name='model_linear')
model2 = ComputationalModel(n_parameters=2, runmodel_object=runmodel5, prior=prior2,
                            error_covariance=np.ones(50), name='model_quadratic')
model3 = ComputationalModel(n_parameters=3, runmodel_object=runmodel6, prior=prior3,
                            error_covariance=np.ones(50), name='model_cubic')

candidate_models= [model1, model2, model3]

# Solution
from UQpy.inference import BayesParameterEstimation
from UQpy.inference import BayesModelSelection
from UQpy.sampling import MetropolisHastings
proposals = [Normal(0.1),
            JointIndependent([Normal(0.1), Normal(0.1)]),
            JointIndependent([Normal(0.15), Normal(0.1), Normal(0.05)])]

model_prior_means = [[0.], [0., 0.], [0., 0., 0.]]

mh1 = ModifiedMetropolisHastings(jump=10, burn_length=1000, proposal=proposals[0], seed=[0])
mh2 = DRAM(jump=10, burn_length=1000, seed=[0., 1.1])
mh3 = MetropolisHastings(jump=10, burn_length=1000,proposal=proposals[2], seed=[0.9, 2.2, 0.])

samplers=[mh1, mh2, mh3]
estimators = []
for i in range(3):
    estimators.append(BayesParameterEstimation(inference_model=candidate_models[i], data=data3, 
                                               sampling_class=samplers[i]))

selection = BayesModelSelection(parameter_estimators=estimators,
                                prior_probabilities=[1. / 3., 1. / 3., 1. / 3.],
                                nsamples=[2000, 2000, 2000])

sorted_indices = np.argsort(selection.probabilities)[::-1]
print('Sorted models:')
print([selection.candidate_models[i].name for i in sorted_indices])
print('Posterior probabilities of sorted models:')
print([selection.probabilities[i] for i in sorted_indices])

Posterior probabilities of sorted models:
[4.64507579525117e-115, 0.0, 1.0]


# Activity 4


## Perform ${BayesParameterEstimation}$ for the Hugoniot relationships.

The model consists of the Rankine-Hugoniot equations. These equations describe the relationship between the states on both sides of a shock wave and express the conservation of mass, momentum and energy:

   \begin{align*}
    & \rho_0 U_s = \rho_1(U_s -u_p)\\
    & P_1=\rho_0 U_s u_p \\
    & E_1 -E_0=\frac{1}{2}(P_1+P_0)(\frac{1}{\rho_0}-\frac{1}{\rho_1})
   \end{align*}

given the relationship between $U_s, u_p$ can be computed and subsequently the quantities $\rho_1, P_1$ and $E_1$ can be computed. We know that the relationship between the shock velocity $U_s$ and particle velocity $u_p$ is given by a cubic polynomial expression as follows:

   \begin{align*}
    U_s = a_0 + a_1 \cdot u_p + a_2 \cdot u_p^2+ a_3 \cdot u_p^3
   \end{align*}


Given the file $\texttt{experimental}\_\texttt{hugoniot}\_\texttt{data.pkl}$ that contains experimental results, perform $\texttt{BayesParameterEstimation}$ to determine the actual distribution of the polynomial parameters that dictate the relationship between the shock $U_s$ and particle velocities $u_p$.

In [ ]:
import pickle
import matplotlib.pyplot as plt

output = open('activity_data.pkl', 'rb')
activity_data = pickle.load(output)
output.close()

In [ ]:
from UQpy.run_model.model_execution.PythonModel import PythonModel
from UQpy.run_model.RunModel import RunModel
model = PythonModel(model_script='activity_model.py',
                    model_object_name='model_quadratic',
                    var_names=['a_0', 'a_1', 'a_2', 'a_3'])
h_func = RunModel(model=model)

In [ ]:
from UQpy.distributions import Normal, JointIndependent
p0 = Normal()
p1 = Normal()
p2 = Normal()
p3 = Normal()
prior = JointIndependent(marginals=[p0, p1, p2, p3])

error_covariance = 1.0

from UQpy.inference import ComputationalModel
inference_model = ComputationalModel(n_parameters=4, runmodel_object=h_func, error_covariance=error_covariance, prior=prior)

In [ ]:
from UQpy.sampling import MetropolisHastings

proposal = JointIndependent([Normal(), Normal(), 
                             Normal(), Normal()])

mh1 = MetropolisHastings(jump=10, burn_length=0, proposal=proposal, seed=[0.0, 0.0, 0.0, 0.0])

In [ ]:
from UQpy.inference import BayesParameterEstimation
bayes_estimator = BayesParameterEstimation(inference_model=inference_model,
                                           data=activity_data, sampling_class=mh1,
                                           nsamples=500)
s = bayes_estimator.sampler.samples

In [ ]:
from sklearn.neighbors import KernelDensity  # for the plots
import matplotlib.pyplot as plt

def pdf_from_kde(domain, samples1d):
    bandwidth = 1.06 * np.std(samples1d) * samples1d.size ** (-1 / 5)
    kde = KernelDensity(bandwidth=bandwidth).fit(samples1d.reshape((-1, 1)))
    log_dens = kde.score_samples(domain)
    return np.exp(log_dens)

In [ ]:
import numpy as np
fig, ax = plt.subplots(1, 4, figsize=(10, 4))

domain = np.linspace(-4, 10, 200)[:, np.newaxis]
pdf_ = pdf_from_kde(domain, s[:, 0])
ax[0].plot(domain, pdf_, label='posterior')
ax[0].plot(domain, p0.pdf(domain), label='prior')
ax[0].set_title('posterior pdf of a_{0}')

domain = np.linspace(-4, 4, 200)[:, np.newaxis]
pdf_ = pdf_from_kde(domain, s[:, 1])
ax[1].plot(domain, pdf_, label='posterior')
ax[1].plot(domain, p1.pdf(domain), label='prior')
ax[1].set_title('posterior pdf of a_{1}')

domain = np.linspace(-4, 4, 200)[:, np.newaxis]
pdf_ = pdf_from_kde(domain, s[:, 2])
ax[2].plot(domain, pdf_, label='posterior')
ax[2].plot(domain, p2.pdf(domain), label='prior')
ax[2].set_title('posterior pdf of a_{2}')

domain = np.linspace(-4, 4, 200)[:, np.newaxis]
pdf_ = pdf_from_kde(domain, s[:, 3])
ax[3].plot(domain, pdf_, label='posterior')
ax[3].plot(domain, p2.pdf(domain), label='prior')
ax[3].set_title('posterior pdf of a_{3}')

plt.show()